In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
data = pd.read_csv('data/Combined_News_DJIA.csv')

In [5]:
data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [6]:
data.shape

(1989, 27)

In [7]:
data.Label.value_counts() # most of the trends are upward

1    1065
0     924
Name: Label, dtype: int64

In [8]:
doc_rating = data[['Label', 'Top1']]

In [9]:
doc_rating.head()

,Label,Top1
0,0,"b""Georgia 'downs two Russian warplanes' as cou..."
1,1,b'Why wont America and Nato help us? If they w...
2,0,b'Remember that adorable 9-year-old who sang a...
3,0,b' U.S. refuses Israel weapons to attack Iran:...
4,1,b'All the experts admit that we should legalis...


# Predict DJI Trend using News

In [10]:
# !pip install gensim
# !pip install pyLDAvis
# nltk.download('stopwords')
# nltk.download('wordnet')
import nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

import gensim
from gensim import corpora
from gensim.models import CoherenceModel

# Plotting tools

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized


### Cleaning and Preprocessing
Remove the punctuations, stopwords and normalize the corpus.

In [12]:
doc_rating['clean_doc'] = doc_rating['Top1'].astype(str).apply(lambda x: clean(x).split())

/Users/xmpuspus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
doc_rating.head()

,Label,Top1,clean_doc
0,0,"b""Georgia 'downs two Russian warplanes' as cou...","[bgeorgia, down, two, russian, warplane, count..."
1,1,b'Why wont America and Nato help us? If they w...,"[bwhy, wont, america, nato, help, u, wont, hel..."
2,0,b'Remember that adorable 9-year-old who sang a...,"[bremember, adorable, 9yearold, sang, opening,..."
3,0,b' U.S. refuses Israel weapons to attack Iran:...,"[b, u, refuse, israel, weapon, attack, iran, r..."
4,1,b'All the experts admit that we should legalis...,"[ball, expert, admit, legalise, drug]"


### Preparing Document-Term Matrix

In [14]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_rating['clean_doc'])

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_rating['clean_doc']]
# doc_term_matrix

In [15]:
# save dictionary
dictionary.save('models/dictionary.sav')

In [14]:
# Load saved dictionary
# dictionary = corpora.Dictionary.load('models/dictionary.sav')
# doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_rating['clean_doc']]

In [16]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=50, random_state=42)

In [17]:
# save lda model
ldamodel.save("models/lda_model.sav")

In [17]:
# load model
# lda_load_model = gensim.models.ldamodel.LdaModel.load("models/lda_model.sav")

# print(lda_load_model.print_topics(num_topics=3, num_words=2))

In [18]:
# List out document topics
lists = ldamodel.get_document_topics(doc_term_matrix, minimum_probability=0.0)

In [19]:
# Convert to dataframe
document_topic_dist = pd.DataFrame([i for i in lists]).apply(lambda x: [i[1] for i in x])

In [20]:
document_topic_dist.head()

,0,1,2,3,4,5,6,7,8,9
0,0.010003,0.010003,0.010004,0.010004,0.010003,0.909967,0.010004,0.010005,0.010004,0.010003
1,0.007696,0.576378,0.007697,0.007696,0.362053,0.007697,0.007695,0.007696,0.007697,0.007696
2,0.011116,0.011118,0.011116,0.011116,0.011117,0.899951,0.011117,0.011116,0.011116,0.011117
3,0.011116,0.011118,0.011118,0.402899,0.508165,0.011117,0.011117,0.011117,0.011117,0.011117
4,0.016673,0.016673,0.016674,0.016672,0.016676,0.016672,0.016677,0.016672,0.016674,0.849936


### Regression

In [32]:
# from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

In [28]:
X = pd.concat([document_topic_dist], axis=1) 
y = doc_rating['Label']

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [33]:
model = RandomForestClassifier(300)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.45      0.42      0.43       273
           1       0.54      0.57      0.56       324

    accuracy                           0.50       597
   macro avg       0.50      0.50      0.50       597
weighted avg       0.50      0.50      0.50       597



In [36]:
# save model
joblib.dump(model, 'models/lr_model.sav')

['models/lr_model.sav']

# Test Full Stack

In [37]:
import pandas as pd
import joblib

In [38]:
import nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

import gensim
from gensim import corpora
from gensim.models import CoherenceModel

In [41]:
# This is the text input
text = 'Trump Sucks'


In [42]:
# Function to Clean text
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [43]:
# Actually clean text
text_process = clean(text).split()
text_process

['trump', 'suck']

In [44]:
# Load dictionary 
loaded_dictionary = corpora.Dictionary.load('models/dictionary.sav') 

# Convert list of words to document term array
doc_term = loaded_dictionary.doc2bow(text_process)

In [45]:
# Load LDA model
lda_load_model = gensim.models.ldamodel.LdaModel.load("models/lda_model.sav")

# Get document topic probabilities
lists = lda_load_model.get_document_topics(doc_term, minimum_probability=0.0)

In [46]:
# Convert to array/list
document_topic =  [i[1] for i in lists]
document_topic

[0.033349402,
 0.033349402,
 0.033354174,
 0.033349402,
 0.033349402,
 0.033349402,
 0.033349402,
 0.6998506,
 0.033349402,
 0.033349402]

In [48]:
# Load classification model
classification_model = joblib.load('models/lr_model.sav')

In [50]:
# predict on output document topic
prediction = classification_model.predict([document_topic])[0]
prediction

1

In [31]:
!pipreqs --force .

INFO: Successfully saved requirements file in ./requirements.txt
